In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import contingency_matrix 


In [2]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [3]:
predicted_test = test.copy()

feature_1= 'curvature_1'


models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_2/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    X=np.array([angle_1]).T
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    

In [4]:
print(predicted_class)
print(true_label)

print(len(predicted_class))
print(len(true_label))

[4, 1, 9, 9, 5, 8, 9, 4, 1, 0, 8, 1, 4, 2, 6, 4, 5, 0, 1, 1, 8, 0, 3, 0, 5, 8, 1, 4, 5, 8, 7, 2, 2, 1, 0, 4, 8, 4, 1, 8, 2, 9, 9, 3, 4, 7, 5, 3, 8, 8, 7, 1, 3, 0, 3, 4, 0, 7, 6, 8, 9, 6, 0, 9, 1, 3, 5, 8, 5, 7, 2, 2, 6, 2, 3, 3, 4, 3, 0, 0, 9, 3, 1, 8, 3, 6, 1, 1, 1, 8, 8, 1, 8, 4, 1, 2, 8, 2, 3, 7, 3, 8, 0, 8, 4, 1, 2, 1, 1, 1, 4, 5, 5, 1, 3, 2, 1, 2, 2, 1, 9, 2, 3, 6, 0, 9, 1, 3, 1, 0, 8, 2, 1, 1, 3, 6, 0, 4, 6, 9, 3, 1, 2, 7, 4, 8, 1, 3, 8, 7, 1, 8, 1, 8, 4, 3, 4, 0, 2, 5, 3, 0, 1, 4, 4, 0, 8, 8, 0, 1, 4, 1, 8, 9, 8, 6, 0, 8, 5, 0, 3, 1, 4, 3, 6, 0, 3, 0, 3, 1, 6, 9, 1, 5, 8, 1, 8, 3, 9, 4, 6, 6, 0, 1, 4, 9, 1, 0, 5, 5, 0, 6, 0, 4, 1, 4, 0, 7, 0, 3, 9, 2, 2, 1, 5, 1, 4, 9, 6, 2, 1, 4, 7, 0, 6, 6, 9, 1, 0, 8, 0, 8, 7, 3, 4, 2, 8, 0]
[4, 1, 9, 3, 5, 8, 0, 4, 6, 0, 8, 4, 7, 2, 6, 4, 7, 3, 1, 2, 8, 0, 9, 0, 5, 3, 6, 4, 5, 2, 4, 5, 2, 0, 0, 4, 2, 1, 2, 9, 3, 2, 9, 3, 1, 7, 5, 5, 8, 6, 7, 4, 5, 3, 3, 1, 0, 9, 6, 8, 9, 0, 6, 9, 9, 8, 1, 0, 2, 4, 5, 2, 6, 9, 3, 4, 4, 2, 6, 0, 9, 9, 8, 7, 2,

In [5]:
matrix=contingency_matrix(true_label, predicted_class)

In [6]:
matrix

array([[21,  3,  0,  0,  0,  0,  5,  0,  4,  1],
       [ 0, 13,  0,  0,  7,  1,  0,  1,  1,  0],
       [ 1,  4,  8,  6,  0,  1,  0,  0,  2,  1],
       [ 2,  3,  3,  8,  0,  1,  1,  1,  1,  2],
       [ 0,  6,  0,  1, 15,  0,  0,  3,  0,  0],
       [ 1,  4,  2,  3,  1,  8,  1,  0,  3,  2],
       [ 6,  3,  3,  1,  0,  0,  9,  0,  3,  0],
       [ 0,  5,  1,  0,  5,  3,  0,  5,  1,  1],
       [ 1,  1,  0,  4,  0,  1,  1,  0, 16,  0],
       [ 0,  3,  4,  5,  0,  0,  0,  1,  2, 11]])

In [7]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.4596774193548387


In [8]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.7983870967741935
